# Generate HuggingFace Dataset

## Imports

In [1]:
import os

import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, load_from_disk

from sentence_transformers import SentenceTransformer

In [2]:
from readability_scores import readability_scores
from lexical_density import lexical_density as Lexical_Density
from avg_word_sentence_length import average_word_length, average_sentence_length
from introduce_typo import introduce_typo
from TF_IDF import calc_tf_idf

## Load dataset

### Loading prompts train csv

In [3]:
prompts_train = pd.read_csv('./data/prompts_train.csv')
prompts_train

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


### Loading summaries train csv

In [4]:
summaries_train = pd.read_csv('./data/summaries_train.csv')
summaries_train.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


## Readability Scores

In [5]:
text = """Although the algorithmic approach to readability has been criticized for not taking into account 
         the deeper, more fluid aspects of comprehension, such as motivation, background knowledge, 
         and personal interest, it's a useful tool for getting a general gauge on text's complexity."""

FK_grade, Gunning_Fog, SMOG = readability_scores(text)
print(f"Flesch-Kincaid Grade Level: {FK_grade:.2f}")
print(f"Gunning Fog Index: {Gunning_Fog:.2f}")
print(f"SMOG Index: {SMOG:.2f}")

Flesch-Kincaid Grade Level: 23.03
Gunning Fog Index: 25.32
SMOG Index: 21.19


In [6]:
text = "The quick brown fox jumps over the lazy dog"
noisy_text = introduce_typo(text, probability=0.2)
print(noisy_text)

The quick pbrown fox jumps over the lazy dog


## Lexical density

In [7]:
# Example list of stop words. You can expand this list as needed.
text = "The cat sat on the mat. The dog sat beside the cat."

print(f"Lexical Density: {Lexical_Density(text)}")

Lexical Density: 0.8571428571428571


## avg_word_sentence_length

In [8]:
text = "Hello world! This is a test. How long is the average sentence? Let's find out."

print(f"Average Word Length: {average_word_length(text):.2f}")
print(f"Average Sentence Length (in words): {average_sentence_length(text):.2f}")

Average Word Length: 5.09
Average Sentence Length (in words): 5.50


## TF-IDF

In [9]:
data1 = {
    'question': ['What is the theme of the text?'],
    'text': ["The short story revolves around a young girl's struggle during the Great Depression."]
}
data2 = {
    'question': ['Where did the event occur?'],
    'text': [ "The documentary narrates events in a small town in Spain during the 1800s."]
}
data3 = {
    'question': ['Who is the main character?'],
    'text': ["The novel centers on the adventures of a sailor named Odysseus."]
}

In [10]:
data={
    'question': prompts_train.loc[0]['prompt_question'],
    'text': prompts_train.loc[0]['prompt_text']
}

In [11]:
data

{'question': 'Summarize at least 3 elements of an ideal tragedy, as described by Aristotle.',
 'text': 'Chapter 13 \r\nAs the sequel to what has already been said, we must proceed to consider what the poet should aim at, and what he should avoid, in constructing his plots; and by what means the specific effect of Tragedy will be produced. \r\nA perfect tragedy should, as we have seen, be arranged not on the simple but on the complex plan. It should, moreover, imitate actions which excite pity and fear, this being the distinctive mark of tragic imitation. It follows plainly, in the first place, that the change of fortune presented must not be the spectacle of a virtuous man brought from prosperity to adversity: for this moves neither pity nor fear; it merely shocks us. Nor, again, that of a bad man passing from adversity to prosperity: for nothing can be more alien to the spirit of Tragedy; it possesses no single tragic quality; it neither satisfies the moral sense nor calls forth pity 

In [12]:
word_score_df = calc_tf_idf(data['question'], data['text'], 10)
word_score_df

['tragedy',
 'tragic',
 'fear',
 'pity',
 'good',
 'like',
 'bad',
 'man',
 'best',
 'stage']

## BM25

In [13]:
class BM25:
    def __init__(self, corpus, k1=1.5, b=0.75):
        self.corpus = corpus
        self.k1 = k1
        self.b = b
        self.doc_lengths = [len(doc.split()) for doc in corpus]
        self.avgdl = sum(self.doc_lengths) / len(corpus)

    def score(self, query, doc):
        """
        Compute BM25 score for a given document and query.
        """
        doc_tokens = doc.split()
        query_tokens = query.split()
        score = 0.0
        
        for token in query_tokens:
            f = doc_tokens.count(token)
            score += (f * (self.k1 + 1)) / (f + self.k1 * (1 - self.b + self.b * len(doc_tokens) / self.avgdl))
            
        return score

    def rank(self, query):
        """
        Rank documents with respect to the given query.
        """
        scores = [(i, self.score(query, doc)) for i, doc in enumerate(self.corpus)]
        return sorted(scores, key=lambda x: x[1], reverse=True)


# Example
corpus = [
    "The cat sat on the mat",
    "The dog sat on the log",
    "Cats and dogs are animals"
]

bm25 = BM25(corpus)
query = "cat on mat"
ranking = bm25.rank(query)
print(ranking)


[(0, 2.922636103151863), (1, 0.9742120343839542), (2, 0.0)]


In [14]:
import re

query, text = data['question'], data['text']
# Splitting the provided text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text.replace('\r\n', ' '))
# Applying BM25 to rank sentences
bm25 = BM25(sentences)
ranking = bm25.rank(query)
# Getting top 3 ranked sentences
top_sentences = [sentences[i[0]] for i in ranking[:3]]

for i, sentence in enumerate(top_sentences, 1):
    print(f"{i}. {sentence}\n")

1. A tragedy, then, to be perfect according to the rules of art, should be of this construction.

2. It should come about as the result not of vice, but of some great error or frailty, in a character either such as we have described, or better rather than worse.

3. It is accounted the best because of the weakness of the spectators; for the poet is guided in what he writes by the wishes of his audience.



## Generate HF Dataset

### Load sentence transformers model

In [14]:
model = SentenceTransformer('all-mpnet-base-v2')

### Preprocess prompts dataset

In [15]:
def preprocess_prompts(prompts_df: pd.DataFrame) -> pd.DataFrame:
    # init an empty new  dataFrame
    new_data = pd.DataFrame({
        'prompt_id': [],
        'prompt_question': [],
        # 'prompt_title': [],
        'prompt_text': []
    })

    for index, row in prompts_df.iterrows():
        # retrieve columns
        prompt_id = row['prompt_id']
        prompt_question = row['prompt_question']
        # prompt_title = row['prompt_title'] # we do not need the title
        prompt_text = row['prompt_text']

        # we are creating a batch of the sentences we want to get embeddings
        sentences = [prompt_question, prompt_text]

        # calling model embedding
        embeddings = model.encode(sentences)

        # Create a new row
        new_row = pd.DataFrame({'prompt_id': [prompt_id], 'prompt_question': [embeddings[0]], 'prompt_text': [embeddings[1]]})
        
        # Append the row
        new_data = pd.concat([new_data.loc[:], new_row], ignore_index=True)

    path = f"./data/preprocessed_data"
    os.makedirs(path, exist_ok=True)
    new_data.to_pickle(f"{path}/prompts_train.pickle")
    return new_data

In [16]:
preprocessed_prompts_train = preprocess_prompts(prompts_train)
preprocessed_prompts_train

,prompt_id,prompt_question,prompt_text
0,39c16e,"[0.0091599105, -0.029818784, 0.0031730002, 0.0...","[-0.010795923, 0.025710443, -0.015630066, 0.01..."
1,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067..."
2,814d6b,"[0.054656938, -0.06503344, 0.011860348, -0.053...","[0.04722372, 0.06517054, 0.00032050064, -0.025..."
3,ebad26,"[0.0067965747, 0.07661588, -0.009160591, 0.033...","[-0.013152148, 0.059951786, -0.0068048476, -0...."


### Preprocess summaries dataset

In [17]:
def normalize_value(x, min=-2, max=4):
    return 2*((x-min)/(max-min))-1

In [18]:
def preprocess_summaries(summaries_df: pd.DataFrame, prompts_df: pd.DataFrame) -> pd.DataFrame :
    # init an empty new  dataFrame
    new_data = pd.DataFrame({
        'student_id': [],
        'prompt_id': [],
        'prompt_question': [],
        'prompt_text': [],
        'text': [],
        'fk_grade': [],
        'gunning_fog': [],
        'smog': [],
        'lexical_density': [],
        'avg_word_length': [],
        'avg_sentence_length': [],
        'content': [],
        'wording': [],
        'normalized_content': [],
        'normalized_wording': []
    })
    
    for index, row in summaries_df.iterrows():
        print(f"\r{index+1}/{len(summaries_df)}", end="")
        # retrieve columns
        student_id = row["student_id"]
        prompt_id = row["prompt_id"]
        text = row["text"]
        content = row["content"]
        wording = row["wording"]
        FK_grade, Gunning_Fog, SMOG = readability_scores(text)
        lexical_density = Lexical_Density(text)
        avg_word_length = average_word_length(text)
        avg_sentence_length = average_sentence_length(text)
        normalized_content = normalize_value(content)
        normalized_wording = normalize_value(wording)

        # we are creating a batch of the sentences we want to get embeddings
        sentences = [text]

        # calling model embedding
        embeddings = model.encode(sentences)

        prompt_row = prompts_df.loc[prompts_df['prompt_id'] == prompt_id]
        prompt_question, prompt_text = prompt_row['prompt_question'].item(), prompt_row['prompt_text'].item()

        # Create a new row
        new_row = pd.DataFrame({
            'student_id': [student_id],
            'prompt_id': [prompt_id],
            'prompt_question': [prompt_question],
            'prompt_text': [prompt_text],
            'text': [embeddings[0]],
            'fk_grade': [FK_grade],
            'gunning_fog': [Gunning_Fog],
            'smog': [SMOG],
            'lexical_density': [lexical_density],
            'avg_word_length': [avg_word_length],
            'avg_sentence_length': [avg_sentence_length],
            'content': [content],
            'wording': [wording],
            'normalized_content': [normalized_content],
            'normalized_wording': [normalized_wording]
        })
        
        # Append the row
        new_data = pd.concat([new_data.loc[:], new_row], ignore_index=True)

    path = f"./data/preprocessed_data"
    os.makedirs(path, exist_ok=True)
    new_data.to_pickle(f"{path}/summaries_train.pickle") 
    return new_data

In [19]:
preprocess_summaries_train = preprocess_summaries(summaries_train, preprocessed_prompts_train)
preprocess_summaries_train

7165/7165

,student_id,prompt_id,prompt_question,prompt_text,text,fk_grade,gunning_fog,smog,lexical_density,avg_word_length,avg_sentence_length,content,wording,normalized_content,normalized_wording
0,000e8c3c7ddb,814d6b,"[0.054656938, -0.06503344, 0.011860348, -0.053...","[0.04722372, 0.06517054, 0.00032050064, -0.025...","[0.041094355, -0.006460443, 0.008264857, -0.06...",9.002500,11.466667,11.208143,0.875000,5.312500,12.000000,0.205683,0.380538,-0.264772,-0.206487
1,0020ae56ffbf,ebad26,"[0.0067965747, 0.07661588, -0.009160591, 0.033...","[-0.013152148, 0.059951786, -0.0068048476, -0....","[0.033534568, 0.112103865, 0.013419032, 0.0166...",5.524444,7.200000,3.129100,0.805556,4.166667,18.000000,-0.548304,0.506755,-0.516101,-0.164415
2,004e978e639e,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[0.022079999, 0.030259458, 0.016786981, 0.0001...",9.238716,11.572072,11.374739,0.731183,5.456989,15.416667,3.128928,4.231226,0.709643,1.077075
3,005ab0199905,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[0.041241955, -0.01394414, 0.015658101, 0.0315...",4.160522,7.057391,7.554174,0.800000,6.200000,4.600000,-0.210614,-0.471415,-0.403538,-0.490472
4,0070c9e7af47,814d6b,"[0.054656938, -0.06503344, 0.011860348, -0.053...","[0.04722372, 0.06517054, 0.00032050064, -0.025...","[0.008337094, 0.04310772, 0.0132706165, -0.052...",10.254643,14.071429,12.540901,0.787500,5.712500,12.307692,3.272894,3.219757,0.757631,0.739919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,"[0.0067965747, 0.07661588, -0.009160591, 0.033...","[-0.013152148, 0.059951786, -0.0068048476, -0....","[0.035653163, 0.043147907, -0.030623915, -0.00...",12.250000,15.015385,13.023867,0.823529,5.000000,26.000000,0.205683,0.380538,-0.264772,-0.206487
7161,ffc34d056498,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[-0.035234254, 0.033364758, 0.02028155, 0.0068...",7.304138,6.625287,7.793538,0.862069,6.034483,7.250000,-0.308448,0.048171,-0.436149,-0.317276
7162,ffd1576d2e1b,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[0.022816774, 0.011155369, -0.0006411369, 0.02...",5.835000,7.333333,7.793538,0.923077,5.000000,13.333333,-1.408180,-0.493603,-0.802727,-0.497868
7163,ffe4a98093b2,39c16e,"[0.0091599105, -0.029818784, 0.0031730002, 0.0...","[-0.010795923, 0.025710443, -0.015630066, 0.01...","[0.012417577, -0.052821647, -0.0033073293, -0....",6.465116,9.454264,9.725611,0.813953,4.627907,14.333333,-0.393310,0.627128,-0.464437,-0.124291


In [20]:
print("fk_grade: ", preprocess_summaries_train['fk_grade'].min(), preprocess_summaries_train['fk_grade'].max())
print("gunning_fog: ", preprocess_summaries_train['gunning_fog'].min(), preprocess_summaries_train['gunning_fog'].max())
print("smog: ", preprocess_summaries_train['smog'].min(), preprocess_summaries_train['smog'].max())
print("lexical_density: ", preprocess_summaries_train['lexical_density'].min(), preprocess_summaries_train['lexical_density'].max())
print("avg_word_length: ", preprocess_summaries_train['avg_word_length'].min(), preprocess_summaries_train['avg_word_length'].max())
print("avg_sentence_length: ", preprocess_summaries_train['avg_sentence_length'].min(), preprocess_summaries_train['avg_sentence_length'].max())

fk_grade:  0.5360526315789471 52.19131782945736
gunning_fog:  1.8 55.010852713178295
smog:  3.1291 30.526468669271875
lexical_density:  0.33181818181818185 1.0
avg_word_length:  3.4782608695652173 8.057692307692308
avg_sentence_length:  3.111111111111111 129.0


-----

In [21]:
specific_prompt_id = '814d6b'
index_of_max_content = preprocess_summaries_train[preprocess_summaries_train['prompt_id'] == specific_prompt_id]['content'].idxmax()
preprocess_summaries_train.loc[index_of_max_content]

student_id                                                  1c50ba91ffba
prompt_id                                                         814d6b
prompt_question        [0.054656938, -0.06503344, 0.011860348, -0.053...
prompt_text            [0.04722372, 0.06517054, 0.00032050064, -0.025...
text                   [0.048191037, 0.05854288, -0.0014909966, -0.04...
fk_grade                                                       16.093193
gunning_fog                                                    18.420168
smog                                                           16.263093
lexical_density                                                 0.767857
avg_word_length                                                  6.14881
avg_sentence_length                                            24.142857
content                                                         3.711374
wording                                                         4.064103
normalized_content                                 

----

In [22]:
preprocess_summaries_train = pd.read_pickle("./data/preprocessed_data/summaries_train.pickle")

In [23]:
preprocess_summaries_train

,student_id,prompt_id,prompt_question,prompt_text,text,fk_grade,gunning_fog,smog,lexical_density,avg_word_length,avg_sentence_length,content,wording,normalized_content,normalized_wording
0,000e8c3c7ddb,814d6b,"[0.054656938, -0.06503344, 0.011860348, -0.053...","[0.04722372, 0.06517054, 0.00032050064, -0.025...","[0.041094355, -0.006460443, 0.008264857, -0.06...",9.002500,11.466667,11.208143,0.875000,5.312500,12.000000,0.205683,0.380538,-0.264772,-0.206487
1,0020ae56ffbf,ebad26,"[0.0067965747, 0.07661588, -0.009160591, 0.033...","[-0.013152148, 0.059951786, -0.0068048476, -0....","[0.033534568, 0.112103865, 0.013419032, 0.0166...",5.524444,7.200000,3.129100,0.805556,4.166667,18.000000,-0.548304,0.506755,-0.516101,-0.164415
2,004e978e639e,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[0.022079999, 0.030259458, 0.016786981, 0.0001...",9.238716,11.572072,11.374739,0.731183,5.456989,15.416667,3.128928,4.231226,0.709643,1.077075
3,005ab0199905,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[0.041241955, -0.01394414, 0.015658101, 0.0315...",4.160522,7.057391,7.554174,0.800000,6.200000,4.600000,-0.210614,-0.471415,-0.403538,-0.490472
4,0070c9e7af47,814d6b,"[0.054656938, -0.06503344, 0.011860348, -0.053...","[0.04722372, 0.06517054, 0.00032050064, -0.025...","[0.008337094, 0.04310772, 0.0132706165, -0.052...",10.254643,14.071429,12.540901,0.787500,5.712500,12.307692,3.272894,3.219757,0.757631,0.739919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,"[0.0067965747, 0.07661588, -0.009160591, 0.033...","[-0.013152148, 0.059951786, -0.0068048476, -0....","[0.035653163, 0.043147907, -0.030623915, -0.00...",12.250000,15.015385,13.023867,0.823529,5.000000,26.000000,0.205683,0.380538,-0.264772,-0.206487
7161,ffc34d056498,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[-0.035234254, 0.033364758, 0.02028155, 0.0068...",7.304138,6.625287,7.793538,0.862069,6.034483,7.250000,-0.308448,0.048171,-0.436149,-0.317276
7162,ffd1576d2e1b,3b9047,"[-0.012024703, 0.016922273, 0.015488309, 0.002...","[0.058067385, 0.042484563, 0.012715496, 0.0067...","[0.022816774, 0.011155369, -0.0006411369, 0.02...",5.835000,7.333333,7.793538,0.923077,5.000000,13.333333,-1.408180,-0.493603,-0.802727,-0.497868
7163,ffe4a98093b2,39c16e,"[0.0091599105, -0.029818784, 0.0031730002, 0.0...","[-0.010795923, 0.025710443, -0.015630066, 0.01...","[0.012417577, -0.052821647, -0.0033073293, -0....",6.465116,9.454264,9.725611,0.813953,4.627907,14.333333,-0.393310,0.627128,-0.464437,-0.124291


### Serialize and deserialize functions

In [24]:
def serialize_array(arr):
    return arr.tobytes()
    
def deserialize_array(binary_string, dtype, shape):
    return np.frombuffer(binary_string, dtype=dtype).reshape(shape)

In [25]:
def preprocess_data(data):
    data['prompt_question'] = data['prompt_question'].apply(serialize_array)
    data['prompt_text'] = data['prompt_text'].apply(serialize_array)
    data['text'] = data['text'].apply(serialize_array)
    return data

In [98]:
preprocess_summaries_train = preprocess_data(preprocess_summaries_train)

### Converting the pandas dataset to huggingFace dataset

In [99]:
train_dataset = Dataset.from_pandas(preprocess_summaries_train)

In [100]:
dataset = {
    'train': train_dataset,
}

In [101]:
dataset = DatasetDict(dataset)
path = './data/hugging_face/'
os.makedirs(path, exist_ok=True)
dataset.save_to_disk(path)

Saving the dataset (0/1 shards):   0%|          | 0/7165 [00:00<?, ? examples/s]

### Test the HF dataset

In [102]:
# Later you can load the entire DatasetDict back from disk like this
dataset_dict = DatasetDict.load_from_disk(path)

In [103]:
print(dataset_dict)
print()
print(deserialize_array(dataset_dict['train'][0]['prompt_question'], np.float32, (768)).shape)
print()
print(deserialize_array(dataset_dict['train'][0]['prompt_text'], np.float32, (768)).shape)
print()
print(deserialize_array(dataset_dict['train'][0]['text'], np.float32, (768)).shape)
print()
print(dataset_dict['train'][0]['content'])
print()
print(dataset_dict['train'][0]['normalized_content'])

DatasetDict({
    train: Dataset({
        features: ['student_id', 'prompt_id', 'prompt_question', 'prompt_text', 'text', 'content', 'wording', 'normalized_content', 'normalized_wording'],
        num_rows: 7165
    })
})

(768,)

(768,)

(768,)

0.205682506482641

-0.2647724978391196
